In [1]:
# User churn problem from the rel-amazon dataset
# on relbench: https://relbench.stanford.edu/datasets/rel-amazon/#user-churn

# 1) https://demo.kurve.ai
# 2) create graph of rel-amazon data
# 3) assign user as parent node customer with depth 4

In [2]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics

In [18]:
# train cut date of 10/1/15
train_path = 'https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/5/output/user_churn_train'

In [19]:
df = pd.read_parquet(train_path)


In [20]:
df.shape

(1850193, 51)

In [21]:
df.head(2)

,cust_customer_id,cust_customer_name,revi_review_time_min,revi_review_time_max,revi_customer_id_count,revi_rating_avg,revi_rating_sum,revi_rating_min,revi_rating_max,revi_verified_sum,...,revi_3dv4_change,revi_4dv7_change,revi_7dv14_change,revi_14dv30_change,revi_30dv60_change,revi_60dv90_change,revi_90dv180_change,revi_180dv365_change,revi_365dv730_change,revi_product_id_label
0,1500289,Charmingmagick,NaT,NaT,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1500295,S. M. Metzler,NaT,NaT,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [22]:
target = 'revi_product_id_label'

In [23]:
df[target].apply(lambda x: 1 if pd.isnull(x) or x == 0 else 0)

0          0
1          0
2          0
3          0
4          0
          ..
1850188    1
1850189    1
1850190    1
1850191    1
1850192    1
Name: revi_product_id_label, Length: 1850193, dtype: int64

In [24]:
df[target] = df[target].apply(lambda x: 1 if pd.isnull(x) or x == 0 else 0)

In [25]:
df[target].sum()/len(df)

0.8120623091753131

In [8]:
# execute compute graph with
# cut date 1/1/2016
test = pd.read_parquet(train_path)
test[target] = test[target].apply(lambda x: 1 if pd.isnull(x) or x == 0 else 0)

In [9]:
test.shape


(1850193, 51)

In [10]:
test[target].sum()/len(test)

0.7892981975393918

In [11]:
test.head(2)

,cust_customer_id,cust_customer_name,revi_review_time_min,revi_review_time_max,revi_customer_id_count,revi_rating_avg,revi_rating_sum,revi_rating_min,revi_rating_max,revi_verified_sum,...,revi_3dv4_change,revi_4dv7_change,revi_7dv14_change,revi_14dv30_change,revi_30dv60_change,revi_60dv90_change,revi_90dv180_change,revi_180dv365_change,revi_365dv730_change,revi_product_id_label
0,1637109,slownight76,NaT,NaT,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1637320,Amazon Customer,NaT,NaT,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [26]:
stypes = infer_df_stype(test.tail(1000))
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and 'label' not in k
    and not k.startswith('cust_')
]

In [27]:
len(features)

45

In [28]:
features = [f for f in features if f in df.columns]
len(features)

45

In [29]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# -------------------------------------------------
# 1. Split off the *final* test set (once!)
# -------------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,          # 20 % held-out test
    stratify=df[target],
    random_state=42
)

# -------------------------------------------------
# 2. K-Fold CV on the remaining 80 %
# -------------------------------------------------
k = 3                                   # change to 10, etc.
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

fold_aucs = []
test_preds = np.zeros(len(X_test))      # out-of-fold predictions on *test*
oof_preds = np.zeros(len(X_train_full)) # optional: OOF on training data

# CatBoost pools (optional but faster)
train_pool = Pool(X_train_full, y_train_full)  # , cat_features=cat_features)

for fold, (idx_tr, idx_va) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    # -----------------------------------------------------------------
    # 3. Fit on the *training* split of this fold
    # -----------------------------------------------------------------
    mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        use_best_model=True,
    
        # capacity vs regularization
        iterations=2000,
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[0.6,0.4],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
    mdl.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        use_best_model=False,
        verbose=False
    )

    # -------------------------------------------------
    # 4. Validation AUC for this fold
    # -------------------------------------------------
    val_pred = mdl.predict_proba(X_va)[:, 1]
    val_auc  = roc_auc_score(y_va, val_pred)
    fold_aucs.append(val_auc)
    print(f"Fold {fold} validation AUC : {val_auc:.4f}")

    # -------------------------------------------------
    # 5. Accumulate predictions on the *final* test set
    # -------------------------------------------------
    test_preds += mdl.predict_proba(X_test)[:, 1] / k

    # (optional) OOF on training data
    oof_preds[idx_va] = val_pred

# -------------------------------------------------
# 6. Final metrics
# -------------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV AUC : {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"Folds AUC   : {[f'{a:.4f}' for a in fold_aucs]}")

test_auc = roc_auc_score(y_test, test_preds)
print(f"\nFinal test AUC (averaged over {k} folds): {test_auc:.4f}")

# -------------------------------------------------
# 7. (Optional) Refit on the *whole* train_full for deployment
# -------------------------------------------------
final_mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        #use_best_model=True,
    
        # capacity vs regularization
        #iterations=2000,
        iterations=int(mdl.best_iteration_ * 1.1),  # a bit more than best
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[0.6,0.4],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
final_mdl.fit(X_train_full, y_train_full)


=== Fold 1 ===
Fold 1 validation AUC : 0.7036

=== Fold 2 ===
Fold 2 validation AUC : 0.7031

=== Fold 3 ===
Fold 3 validation AUC : 0.7048

=== CV Summary ===
Mean CV AUC : 0.7038 ± 0.0007
Folds AUC   : ['0.7036', '0.7031', '0.7048']

Final test AUC (averaged over 3 folds): 0.7037
0:	total: 103ms	remaining: 2m 16s
200:	total: 40.1s	remaining: 3m 44s
400:	total: 1m 24s	remaining: 3m 15s


KeyboardInterrupt: 

In [32]:
final_mdl.fit(X_train_full, y_train_full)

test['prob'] = final_mdl.predict_proba(test[features])[:,1]

0:	total: 94.2ms	remaining: 2m 4s
200:	total: 40.1s	remaining: 3m 44s
400:	total: 1m 24s	remaining: 3m 15s
600:	total: 2m 9s	remaining: 2m 36s
800:	total: 2m 55s	remaining: 1m 54s
1000:	total: 3m 40s	remaining: 1m 11s
1200:	total: 4m 25s	remaining: 27.6s
1325:	total: 4m 53s	remaining: 0us


In [33]:
metrics.roc_auc_score(test[target], test['prob'])

0.6997150687280074